# KNN

In [2]:
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
import numpy as np
import pandas as pd
import os
import re
import math

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf-8"))

athenaeum_authors_preview.csv
athenaeum_painting_filtered.csv
athenaeum_paintings.csv
athenaeum_paintings_sizes.csv
color_histograms.csv
complete_data.csv
images
images_athenaeum
images_sizes_2325.csv
net_predicted.csv
painter_info_clean.csv
painting_info_clean.csv
resized_200
test_author200.csv
test_data.csv
train_author200.csv
train_data.csv



In [1]:
import neural_net_v4
import data_preparing_v4 as data_preparing


Using TensorFlow backend.


In [3]:
# get the raw images data
train_data = data_preparing.get_not_net_data(painters = 3, train = True)
test_data = data_preparing.get_not_net_data(painters = None, train = False)

Loading images: 739/7392K
Loading images: 439/4392K


In [4]:
# 3*200*200 colors

img_colors = train_data['images_colors']
file_name = train_data['file_name']
short_name = train_data['short_name']

In [5]:
## Prepare data for KNN
X_train, X_val, y_train, y_val, fn_train, fn_val = cross_validation.train_test_split(
                                                                   img_colors,
                                                                   short_name, 
                                                                   file_name,
                                                                   test_size=.2,
                                                                   random_state=777)

print "Length of Validation Set:" + str(len(y_val))
print "Length of Train Set:" + str(len(y_train))
print "Ratio: " + str(len(y_val) / float(len(y_train)))

Length of Validation Set:148
Length of Train Set:591
Ratio: 0.250423011844


In [6]:
## Train the KNN model
print "Training KNN Model..."

## n_jobs=-1: Use all cores
## n_neighbors = Number of Neighbors
n_neighbors = int(math.ceil(math.sqrt(len(img_colors))))
model = KNeighborsClassifier(n_neighbors=n_neighbors,
                             n_jobs=-1)
model.fit(X_train, y_train)

Training KNN Model...


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=28, p=2,
           weights='uniform')

In [7]:
## Predict on the validation set using the KNN model.
validations_predictions = model.predict(X_val)
print validations_predictions

['gogh_van' 'rembrand' 'rembrand' 'gogh_van' 'gogh_van' 'gogh_van'
 'gogh_van' 'rembrand' 'gogh_van' 'rembrand' 'gogh_van' 'rembrand'
 'gogh_van' 'rembrand' 'gogh_van' 'gogh_van' 'rembrand' 'rembrand'
 'gogh_van' 'gogh_van' 'gogh_van' 'gogh_van' 'rembrand' 'rembrand'
 'gogh_van' 'gogh_van' 'gogh_van' 'rembrand' 'rembrand' 'rubens' 'rembrand'
 'gogh_van' 'gogh_van' 'gogh_van' 'rembrand' 'rembrand' 'rembrand'
 'rembrand' 'rembrand' 'rembrand' 'gogh_van' 'gogh_van' 'gogh_van'
 'rembrand' 'gogh_van' 'gogh_van' 'rembrand' 'rembrand' 'gogh_van'
 'gogh_van' 'gogh_van' 'gogh_van' 'gogh_van' 'rembrand' 'rembrand'
 'rembrand' 'rembrand' 'gogh_van' 'gogh_van' 'gogh_van' 'rembrand'
 'gogh_van' 'gogh_van' 'gogh_van' 'gogh_van' 'rembrand' 'rubens' 'gogh_van'
 'gogh_van' 'gogh_van' 'rembrand' 'gogh_van' 'rembrand' 'rembrand'
 'gogh_van' 'rembrand' 'rembrand' 'rembrand' 'gogh_van' 'gogh_van'
 'gogh_van' 'gogh_van' 'gogh_van' 'rembrand' 'gogh_van' 'gogh_van'
 'gogh_van' 'rembrand' 'rembrand' 'gogh_van'

In [12]:
results = pd.DataFrame()

results.insert(0,'actual',y_val)
results.insert(1,'predictions',validations_predictions)

misclassified = (results['actual'] != results['predictions']).mean()
misclassified

0.31756756756756754

In [13]:
## Give me the 5 nearest neighbors of the first two items in the validation set
pd.concat((fn_val.iloc[:5].reset_index(drop = True), 
           pd.DataFrame(model.kneighbors(X_val[0:5], n_neighbors=3, return_distance=False)).
        applymap(lambda x: fn_train.iloc[x])), axis = 1)

,file_name,0,1,2
0,rubens_7graphic_06sketci.jpg,rubens_7graphic_20drawin.jpg,rubens_7graphic_24drawin.jpg,gogh_van_19_1remy06.jpg
1,rembrand_15oldtes_04oldtes.jpg,rembrand_24portra_102portr.jpg,rembrand_14biblic_64newtes.jpg,rembrand_25portra_135portr.jpg
2,rubens_42portra_11family.jpg,rembrand_27self_23sp1669.jpg,gogh_van_02_nuenen16.jpg,rembrand_21portra_06portrb.jpg
3,rubens_7graphic_09euchar.jpg,gogh_van_16_selfpo07.jpg,rubens_7graphic_11sketc1.jpg,rubens_7graphic_24drawin.jpg
4,gogh_van_16_selfpo10.jpg,gogh_van_16_selfpo04.jpg,gogh_van_16_selfpo13.jpg,gogh_van_16_selfpo09.jpg


In [14]:
# accuracy for the validation set
model.score(X_val,y_val)


0.68243243243243246

In [16]:
# accuracy for the trianing set
model.score(X_train,y_train)


0.63451776649746194

In [24]:
test_data_img = test_data['images_colors']
test_labels = test_data['short_name']
test_data_predictions = model.predict(test_data_img)
rslt = test_labels  == test_data_predictions

new_data_df = pd.DataFrame()
new_data_df.insert(0,'actual',test_labels)
new_data_df.insert(1,'predictions',test_data_predictions)
new_data_df.insert(2,'results',rslt)

print "Accuracy of the Model:" + str(model.score(test_data_img, test_labels))

new_data_df

Accuracy of the Model:0.255125284738


,actual,predictions,results
1,renoir,gogh_van,False
2,lippi__filippo,gogh_van,False
3,goya,gogh_van,False
5,rubens,gogh_van,False
6,monet,gogh_van,False
7,mantegna,gogh_van,False
8,leonardo,gogh_van,False
9,caravagg,rembrand,False
10,angelico,gogh_van,False
11,rubens,gogh_van,False


In [25]:
prediction_nearest_neighbors = pd.DataFrame(model.kneighbors(test_data_img, n_neighbors=107, return_distance=False))
prediction_nearest_neighbors

,0,1,2,3,4,5,6,7,8,9,...,97,98,99,100,101,102,103,104,105,106
0,551,156,129,267,108,498,335,210,319,417,...,332,193,370,256,200,131,467,433,585,57
1,98,0,526,387,574,272,170,579,35,385,...,208,519,550,339,411,244,543,78,121,185
2,30,169,589,514,392,562,121,251,329,539,...,356,140,181,184,396,165,452,89,284,125
3,198,277,514,323,382,413,392,281,148,562,...,248,301,563,60,481,424,559,431,29,173
4,210,35,526,0,551,145,344,385,300,134,...,133,367,522,27,291,413,57,51,332,264
5,589,551,67,583,514,467,507,452,131,51,...,127,208,59,264,86,90,378,559,145,265
6,267,551,514,170,210,156,246,168,108,102,...,413,550,279,250,134,131,487,54,392,264
7,557,500,21,179,325,8,468,315,285,523,...,172,408,159,88,302,150,416,278,43,414
8,170,35,427,348,146,526,98,215,320,236,...,378,507,574,116,249,370,483,258,9,160
9,514,551,129,57,62,498,562,507,108,121,...,317,27,550,95,573,273,433,168,146,431


In [50]:
painting_info_clean = pd.read_csv('data/painting_info_clean.csv')
sizes_paintings = pd.read_csv('data/images_sizes_2325.csv')
sizes_paintings = pd.merge(painting_info_clean[['file_name']], sizes_paintings, how = 'left', on='file_name')


# preparing train, test data
complete_data = painting_info_clean.merge(sizes_paintings, how = 'inner', on = 'file_name')
complete_data.to_csv('./data/complete_data.csv', index = False)


In [58]:
complete_data = pd.read_csv('./data/complete_data.csv')
file_list = complete_data['file_name']

In [60]:
nearest_neighbors = list(model.kneighbors(test_data_img[5], n_neighbors=31, return_distance=False)[0])
NN_images = [y_train.iloc[x] for x in nearest_neighbors]
[ (i,NN_images.count(i)) for i in set(NN_images) ]

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[('gogh_van', 26), ('rembrand', 1), ('rubens', 4)]

In [59]:
nearest_neighbors = list(model.kneighbors(test_data_img[5], n_neighbors=107, return_distance=False)[0])

[file_list.iloc[x] for x in nearest_neighbors]

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


['cezanne_4_4still7.jpg',
 'cezanne_5_1variou2.jpg',
 'rubens_7graphic_02sketch.jpg',
 'cezanne_4_4still1.jpg',
 'goya_3_305goya.jpg',
 'goya_6_601goya.jpg',
 'goya_4_417goya.jpg',
 'goya_7_705goya.jpg',
 'renoir_4_4renoi04.jpg',
 'rembrand_91nonrem_16noremb.jpg',
 'rembrand_91nonrem_03noremb.jpg',
 'cezanne_5_4bather3.jpg',
 'goya_4_418goya.jpg',
 'angelico_08_1trinita.jpg',
 'angelico_02_predel2.jpg',
 'angelico_00_11fieso3.jpg',
 'angelico_05_cortona1.jpg',
 'goya_7_720goya.jpg',
 'goya_4_405goya.jpg',
 'angelico_07_marco_p8.jpg',
 'angelico_05_predell3.jpg',
 'angelico_07_panel6.jpg',
 'manet_4_4manet18.jpg',
 'angelico_14_3brisio.jpg',
 'renoir_3_3renoi201.jpg',
 'manet_2_2manet02.jpg',
 'manet_3_3manet04.jpg',
 'angelico_12_70lastju.jpg',
 'renoir_4_4renoi02.jpg',
 'goya_9_blac639.jpg',
 'goya_5_510goya.jpg',
 'manet_1_6stilli3.jpg',
 'monet_08_7liliy03.jpg',
 'rubens_7graphic_11sketcx.jpg',
 'rembrand_91nonrem_01noremb.jpg',
 'monet_08_3bridge2.jpg',
 'manet_3_3manet17.jpg',
 'm